In [8]:
import numpy

In [25]:
class connectanator:
    CONNECT_NUM = 4
    SLOTS = 2 * CONNECT_NUM -1
    ROWS = SLOTS - 1
    MOVES = range(1, SLOTS+1)
    PLAYERS = ("Red", "Yellow")

    def __init__(self):
        self.board = numpy.zeros((self.ROWS, self.SLOTS), dtype=int)
        self.turn = 0

    def get_move(self, case:str="default"):
        players_turn = self.get_whos_turn()
        while True:
            match case:
                case "default":
                    move = input(f"The {players_turn} player puts their counter in slot...")
                case "bad_input":
                    move = input(f"The {players_turn} player didn't realise they had to put their counter in slot 1-{self.SLOTS}. (Type 'exit' to close the game.)")
                case "full":
                    move = input(f"The {players_turn} player tried to put their counter in the full slot {checked_move}, however they get to try again...(Type 'exit' to close the game.)")
                case "all_g":
                    self.move = checked_move
                    break

            if move == 'exit':
                break
            
            try: 
                move = int(move)
            except ValueError:
                case = "bad_input"; continue
            
            if move not in self.MOVES:
                case = "bad_input"
            elif self.board[0, move-1] != 0:
                case = "full"
            else: 
                checked_move = move; case = "all_g"
            

    def place_counter(self):
        col = self.board[:, self.move]
        for i, val in enumerate(col):
            if val != 0:
                self.board[i-1, self.move] = self.turn % 2


    def check_board():
        pass

    def get_whos_turn(self):
        return self.PLAYERS[self.get_player()]
    
    def get_turn(self):
        return self.turn
    
    def get_player(self):
        return self.turn % 2
    

In [ ]:
def main():
    game = connectanator()
    while True:
        game.get_move()
        game.place_counter()
        turn = game.get_turn()
        if turn > 5:
            break

if __name__ == "__main__":
    main()